### Library installation and script setup
This box needs to only be run once. It builds the environment to carry out the rest of the analysis.

In [1]:
# Run one time only - install pip and unusual Libraries
import pip
import pandas as pd
import shapely
from shapely.wkt import loads
from shapely.geometry import box
from shapely.geometry import MultiPolygon
import time
import json
import os, sys
import numpy as np
from gbdxtools import Interface
from gbdxtools.task import env
from gbdxtools import CatalogImage
gbdx = Interface()
%matplotlib inline
import geopandas as gpd
from shapely.geometry import Point

### Set UTM zone of points, number of images to return per AOI

In [2]:
WGS84 = {'init' :'epsg:4326'}
keep = 1 # number of images to return per AOI
test = None # limit the point dataframe to this many objects - test mode

### Define AOI and bounding box of AOI from shapefile
The user needs to have their Area of Interest ("AOI") defined as points, in shapefile format. This should be uploaded to the Jupyter server. This step should be performed before running the next cell. 

The file must contain an additional field 'buffer_m' - which sets the distance around the point, in metres, to be considered as the AOI when searching for matching images

In [3]:
# set file path and point layer names
pointsfile = r'plot.csv'
pth = r'C:\Users\charl\Documents\GOST\WWTP'

# Set up point dataframe
pointframe = pd.read_csv(os.path.join(pth, pointsfile))
pointframe = pointframe[:test]

# Create Geometry object
geometry = [Point(xy) for xy in zip(pointframe.Longitude, pointframe.Latitude)]

# Import points GeoDataFrame
points_list = gpd.GeoDataFrame(pointframe, crs = WGS84, geometry = geometry)

# Ensure starting from WGS84
points_list = points_list.to_crs(WGS84)

# move point geometry to a list
pointz = list(points_list['geometry'])

### Define Categorical Screening Parameters

In [5]:
cutoff_cloud_cover = 0   # images with CC over this threshold discarded
cutoff_overlap = 100     # images with AOI overlap below this threshold discarded. [N.b.: keep small if AOI large.]
cutoff_date_upper = '10-Oct-18'  # images newer than this date discarded
cutoff_date_lower = '10-Oct-14'  # images older than this date discarded\
cutoff_nadir = 25 # Images at nadir angles greater than threshold discarded
cutoff_pan_res = 1 # Images below this resolution discarded
accepted_bands = ['PAN_MS1','PAN_MS1_MS2','PAN'] #  Images with any other band entry discarded
acceptable_sensors = ['WORLDVIEW03_VNIR', 'WORLDVIEW02']

### Define Continuous Image Ranking Preferences

In [6]:
optimal = {
    'optimal_date' : '10-Oct-18', # Optimal date (enter as dd-mmm-yy)
    'optimal_pan_res' : 0.1, # Optimal pan resolution, metres
    'optimal_nadir' : 0 # optimal image angle. 0 = vertical
}

# Define continuous image ranking preference weights. Must sum to 1.
# If user cares more about scenes being contemporaneous, up 'date' weighting at expense of other categories. 

pref_weights = {
    'cloud_cover': 0.3,
    'overlap':0,
    'date': 0.1,
    'nadir': 0.2,
    'resolution': 0.4
    }

### Define Catalog Search Function

In [7]:
#--------------------------------------------------------------------------------------
#Perform Initial Vector Search: AOI Vector Against Image Catalog for All Potential Images
#--------------------------------------------------------------------------------------

# Function queries the DigitalGlobe image database based on the vector 
# footprint of the chosen AOI, and returns a list of catalog IDs which at least partially cover the AOI. 
# As default, it returns up to 1000 images, with a cloud cover less than 25%. 

# Returns up to 1000 images where cloud cover smaller than 25%
def SearchUnordered(bbox, _type, cloud_cover, count):
    aoi = AOI.wkt
    query = "(item_type:WV03_VNIR OR item_type:WV02) AND item_type:DigitalGlobeAcquisition"
    query += " AND attributes.cloudCover_int:<{}".format(cloud_cover)
    return gbdx.vectors.query(aoi, query, count=count)

### Define Function for Generating .csv of all Matching Scenes 

In [8]:
#--------------------------------------------------------------------------------------
### Generate CSV of images that match criteria
#--------------------------------------------------------------------------------------

# Function generates a .csv file with the key metadata about the image which will be used for further processing later. 
# This is the definitive database for the given AOI. No filtering has yet taken place.
# Key calculated statistics include: the area where the scene and the AOI don't overlap (AA), 
# the corresponding overlap with the image (BB), the fraction of the AOI covered (frac), 
# and the binary flag for whether the image is yet in IDAHO. 

def ImageList(ids):
    
    # Define Counters
    l = 0    # number of non-IDAHO images
    scenes = [] # list containing metadata dictionaries of all scenes in our AOI 

    # Toggle for printing images to screen
    download_thumbnails = 0

    # Loop catalog IDs
    for i in ids:

        # Fetch metadata dictionary for each catalog ID in ids list
        r = gbdx.catalog.get(i)

        # Check location of ID - is it in IDAHO?
        location = gbdx.catalog.get_data_location(i)

        # Defines IDAHO variable as binary 1 / 0 depending on whether it is in IDAHO already or not
        if location == 'not_delivered':
            l = l + 1
            idaho = 0
        else:
            idaho = 1

            # Download image if image in IDAHO and toggle on
            if download_thumbnails == 1:
                image = CatalogImage(i, band_type="MS", bbox=bboxx)
                image.plot(w=10, h=10)
            else:
                pass

        # Print statement to consol for key image variables
        print 'ID: %s, Timestamp: %s, Cloud Cover: %s, Image bands: %s, IDAHO: %s' % (i,r['properties']['timestamp'],r['properties']['cloudCover'],r['properties']['imageBands'],idaho)

        # Calculate the percentage overlap with our AOI for each scene
        # load as a Shapely object the wkt representation of the scene footprint
        footprint = r['properties']['footprintWkt']
        shapely_footprint = shapely.wkt.loads(footprint)

        # Calculate the object that represents the difference between the AOI and the scene footprint 
        AA = AOI.difference(shapely_footprint)

        # Define frac as the fraction, between 0 and 1, of the AOI that the scene covers
        frac = 1 - ((AA).area / AOI.area)

        # Create BB - the proxy for the useful area. IF scene entirely contains AOI, then BB = AOI, else it is the intersection 
        # of the scene footprint and the AOI
        BB = AOI 
        if frac < 1:
            BB = AOI - AA
        #shapely_footprint.intersection(AOI)
        # Similarly, AA, the difference area between AOI and the scene, can be set to null if the scene contains 100% of the AOI 
        if frac == 1:
            AA = ""

        # Append key metadata to list obejct 'scenes' for the current scene, as a dictionary. This then moves into the pandas dataframe.
        # Several objects here are from DigitalGlobe's metadata dictionary (anything with an r start)
        scenes.append({
            'ID':i, 
            'TimeStamp':r['properties']['timestamp'],
            'CloudCover':r['properties']['cloudCover'],
            'ImageBands':r['properties']['imageBands'],
            'On_IDAHO':idaho,
            'browseURL': r['properties']['browseURL'],
            'Overlap_%': frac * 100,
            'PanResolution': r['properties']['panResolution'],
            'MultiResolution': r['properties']['multiResolution'],
            'OffNadirAngle': r['properties']['offNadirAngle'],
            'Sensor':r['properties']['sensorPlatformName'],
            'Full_scene_WKT':r['properties']['footprintWkt'],
            'missing_area_WKT':AA,
            'useful_area_WKT':BB
            })

    # Summary Statistics - show totals for images, both in IDAHO and currenlty unavailable images. 
    print 'Number of catalog IDs not available in IDAHO: %s' % l
    print 'Number of catalog IDs available in IDAHO: %s' % (len(ids) - l)

    # Define column order for dataframe of search results
    cols = ['ID','Sensor','ImageBands','TimeStamp','CloudCover','Overlap_%','PanResolution','MultiResolution','OffNadirAngle','On_IDAHO','browseURL','Full_scene_WKT','useful_area_WKT','missing_area_WKT']

    #Generate pandas dataframe from results
    out = pd.DataFrame(scenes,columns = cols)

    #return out
    return out

### Define Categorical Screening Function for Removing Disqualified Images

In [9]:
#--------------------------------------------------------------------------------------
### Categorical Screening Function
#--------------------------------------------------------------------------------------

# Performs pandas .loc operations to cut out scenes which don't meet the criteria specified by the 'cutoff' 
# series of variables.

def CategoricalScreen(out):
    
    # Convert Timestamp field to pandas DateTime object
    out['TS'] = out['TimeStamp'].apply(lambda x: pd.Timestamp(x))

    # Add separate date and time columns for easy interpretation
    string = out['TimeStamp'].str.split('T')
    out['Date'] = string.str.get(0)
    out['Time'] = string.str.get(1)

    # Categorical Search: remove disqualified images. Copy of dataframe taken, renamed to 'out_1stcut'.
    out_1stcut = out.loc[(out['CloudCover'] <= cutoff_cloud_cover) & 
                         (out['Overlap_%'] >= cutoff_overlap) & 
                         (out['TS'] > pd.Timestamp(pd.to_datetime(cutoff_date_lower, utc = True))) & 
                         (out['TS'] < pd.Timestamp(pd.to_datetime(cutoff_date_upper, utc = True))) &
                         (out['ImageBands'].isin(accepted_bands)) & 
                         (out['OffNadirAngle'] <= cutoff_nadir) & 
                         (out['PanResolution'] <= cutoff_pan_res)
                        ]

    # Print to consol remaining images after categorical search undertaken
    print '\nNumber of results remaining: %s' % len(out_1stcut.index)
    
    # Only keep WV02 and WV03
    out_1stcut = out_1stcut.loc[out_1stcut.Sensor.isin(acceptable_sensors)]
    
    # return out_1stcut
    return out_1stcut

### Define Function for Continuous Image Quality Ranking

In [10]:
#--------------------------------------------------------------------------------------
### Image Ranking Function
#--------------------------------------------------------------------------------------
# For the remaining images that met minimum quality thresholds, rank them, best to worst. This is done on a points-based system. Images accrue points for: 
# - every % of cloud cover (1 point)
# - every % of missed overlap with the AOI (1 point)
# - every week away from the optimal date (1 point)
# - every degree away from nadir (1 point)
# - every cm of resolution worse than the optimal resolution 

# User preferences are defined in the 'pref_weights' dictionary. 
# Ensure all weights sum to 1.

def ImageRank(out_1stcut, optimal, pref_weights):
    # Apply ranking method over all non-disqualified search results for each field
    optimal_date = pd.to_datetime(optimal['optimal_date'], utc = True)

    # each 1% of cloud cover = 1 point
    out_1stcut['points_CC'] = (out_1stcut['CloudCover'])  

    # each 1% of overlap missed = 1 point
    out_1stcut['points_Overlap'] = (100 - out_1stcut['Overlap_%'])  

    # each week away from the optimal date = 1 point 
    try:
        out_1stcut['points_Date'] = ((abs(out_1stcut['TS'] - optimal_date)).view('int64') / 60 / 60 / 24 / 1E9) / 7 
    except:
        out_1stcut['points_Date'] = 0
    
    # each degree off nadir = 1 point
    out_1stcut['points_Nadir'] = abs(out_1stcut['OffNadirAngle'] - optimal['optimal_nadir']) 

    # each cm of resolution worse than the optimal resolution = 1 point
    out_1stcut['points_Res'] = (out_1stcut['PanResolution'] - optimal['optimal_pan_res']).apply(lambda x: max(x,0)) * 100 

    # Define ranking algorithm - weight point components defined above by the preference weighting dictionary
    def Ranker(out_1stcut, pref_weights):
        a = out_1stcut['points_CC'] * pref_weights['cloud_cover']
        b = out_1stcut['points_Overlap'] * pref_weights['overlap']
        c = out_1stcut['points_Date'] * pref_weights['date'] 
        d = out_1stcut['points_Nadir'] * pref_weights['nadir']
        e = out_1stcut['points_Res'] * pref_weights['resolution']

        # Score is linear addition of the number of 'points' the scene wins as defined above. More points = worse fit to criteria
        rank = a + b + c + d + e
        return rank

    # Add new column - Rank Result - with the total number of points accrued by the scene 
    out_1stcut['RankResult'] = Ranker(out_1stcut,pref_weights)

    # Add a Preference order column - Pref_Order - based on Rank Result, sorted ascending (best scene first)
    out_1stcut = out_1stcut.sort_values(by = 'RankResult', axis = 0, ascending = True)
    out_1stcut = out_1stcut.reset_index()
    out_1stcut['Pref_order'] = out_1stcut.index + 1
    out_1stcut = out_1stcut.drop(['index'], axis = 1)
    out_2ndcut = out_1stcut

    return out_2ndcut

### Define function for constructing final output file
Before construction of the scene mosaic, output a .csv with the details of selected and ranked scenes, named 'Scene_List.csv'

In [11]:
def OutputBuilder(out_2ndcut, keep):
    
    # trim to just the useful columns
    cols = ['Point','AOI_ID','AOI_WKT','ID','Sensor','ImageBands','Date','Time','CloudCover','Overlap_%','PanResolution','MultiResolution','OffNadirAngle','On_IDAHO','Pref_order','RankResult','points_CC','points_Overlap','points_Date','points_Nadir','points_Res','browseURL','Full_scene_WKT','useful_area_WKT','missing_area_WKT']
    out_2ndcut = out_2ndcut[cols]
    
    # trim to top 10 images by rank per AOI
    out_2ndcut = out_2ndcut[:keep]
    
    return out_2ndcut

### Define Main Function

In [12]:
def Main(AOI, i, keep, pointz): 
    
    print(AOI.centroid)
    
    # Run search on Area of Interest (AOI). Passes in AOI in Well Known Text format (wkt)
    records = SearchUnordered(AOI.wkt, 'DigitalGlobeAcquisition', count = 10, cloud_cover = 25)

    # Create list object of all catalog IDs returned in search
    ids = [r['properties']['attributes']['catalogID'] for r in records]
    
    # Generate matching scene .csv
    out = ImageList(ids)
    
    # Apply categorical screen
    out1stcut = CategoricalScreen(out)
    
    # Continuously Rank remaining images
    out2ndcut = ImageRank(out1stcut, optimal, pref_weights)
    
    # Add Point ID column
    out2ndcut['Point'] = pointz[i-1]
    out2ndcut['AOI_ID'] = i
    out2ndcut['AOI_WKT'] = AOI_list[i-1]
    
    # Clean Output
    finals = OutputBuilder(out2ndcut, keep) 
    
    # Append results to scene_list
    scene_list.append(finals)

### Execute Main for all AOIs

In [13]:
scene_list = []    
i = 1

for AOI in AOI_list:
    Main(AOI, i, keep, pointz)
    i += 1
    
output = pd.concat(scene_list)
outname = 'Output_%s.csv' % keep
output.to_csv(os.path.join(pth, outname))
print('\nProcess complete')

POINT (-5.274526838880768 43.49078029188657)
ID: 103001006C4BAF00, Timestamp: 2017-07-16T11:27:58.000Z, Cloud Cover: 4, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 103001006D688E00, Timestamp: 2017-07-16T11:29:46.000Z, Cloud Cover: 5, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 1030050057F90400, Timestamp: 2017-01-24T11:16:18.959Z, Cloud Cover: 1, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 1040010009701200, Timestamp: 2015-04-07T11:29:06.411Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 1
ID: 10300100279ADE00, Timestamp: 2013-09-02T11:50:09.210Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 103001002773A000, Timestamp: 2013-09-13T11:45:16.324Z, Cloud Cover: 1, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 103001000DA2B400, Timestamp: 2011-09-08T11:50:29.348Z, Cloud Cover: 10, Image bands: PAN_MS1_MS2, IDAHO: 0
Number of catalog IDs not available in IDAHO: 6
Number of catalog IDs available in IDAHO: 1

Number of results remaining: 1
POINT (16.63242480624612 49.12916216525642)
ID: 1030050079F9

ID: 103001006402FB00, Timestamp: 2017-02-10T02:28:50.132Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 1
Number of catalog IDs not available in IDAHO: 8
Number of catalog IDs available in IDAHO: 2

Number of results remaining: 1
POINT (6.975692806558871 4.790837991715865)
ID: 103001007C4A3100, Timestamp: 2018-04-11T10:07:33.000Z, Cloud Cover: 23, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 103001007B1B8200, Timestamp: 2018-04-11T10:06:16.000Z, Cloud Cover: 15, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 1030010078762100, Timestamp: 2018-01-03T10:08:14.000Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 10300100761E4B00, Timestamp: 2018-01-03T10:08:31.000Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 10400100266E2300, Timestamp: 2017-01-01T10:21:20.510Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 10400100357C4B00, Timestamp: 2017-11-27T10:25:54.000Z, Cloud Cover: 23, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 1040010018073400, Timestamp: 2016-02-11T10:09:29.748Z, 

ID: 1040010032BC3B00, Timestamp: 2017-09-28T06:05:57.000Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 10400100195A1600, Timestamp: 2016-03-03T05:43:02.327Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 1040010019523900, Timestamp: 2016-03-15T05:33:34.015Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 104001001D904E00, Timestamp: 2016-05-25T05:54:27.391Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 104001001B0F5D00, Timestamp: 2016-04-16T05:39:26.132Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 1040010016CD5C00, Timestamp: 2016-01-06T05:43:23.236Z, Cloud Cover: 17, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 1040010027039500, Timestamp: 2016-12-22T05:59:49.670Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 0
Number of catalog IDs not available in IDAHO: 10
Number of catalog IDs available in IDAHO: 0

Number of results remaining: 3
POINT (-75.69221569946799 41.38825600778653)
ID: 103001007C315E00, Timestamp: 2018-04-23T15:56:44.000Z, 

ID: 104001003D529B00, Timestamp: 2018-06-02T11:35:07.000Z, Cloud Cover: 17, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 103001007F679E00, Timestamp: 2018-06-02T11:21:33.000Z, Cloud Cover: 21, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 1030010080B00400, Timestamp: 2018-06-23T11:50:40.000Z, Cloud Cover: 9, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 103001007F02B400, Timestamp: 2018-06-23T11:49:20.000Z, Cloud Cover: 9, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 103001006E5E2B00, Timestamp: 2017-08-31T11:33:24.193Z, Cloud Cover: 18, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 103001006A9E4E00, Timestamp: 2017-06-21T11:48:59.000Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 1
ID: 104001001CA28B00, Timestamp: 2016-05-04T11:46:32.862Z, Cloud Cover: 11, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 1040010004940400, Timestamp: 2014-12-03T11:34:11.879Z, Cloud Cover: 1, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 1030050048F8EE00, Timestamp: 2016-02-15T11:07:09.015Z, Cloud Cover: 1, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 10

ID: 10400100233B6E00, Timestamp: 2016-10-09T06:05:43.898Z, Cloud Cover: 23, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 103001000367C000, Timestamp: 2009-12-13T05:27:30.359Z, Cloud Cover: 0, Image bands: PAN, IDAHO: 0
Number of catalog IDs not available in IDAHO: 9
Number of catalog IDs available in IDAHO: 1

Number of results remaining: 3
POINT (-58.44132945895971 -34.68808615292283)
ID: 10400100404B6200, Timestamp: 2018-08-15T14:54:12.000Z, Cloud Cover: 15, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 104001003DC1F400, Timestamp: 2018-06-23T14:10:08.000Z, Cloud Cover: 4, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 10400100416CD200, Timestamp: 2018-08-26T14:31:24.000Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 10400100414FC600, Timestamp: 2018-08-25T14:13:59.000Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 10400100417CFC00, Timestamp: 2018-09-07T14:21:00.000Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 1
ID: 103001007BCA1F00, Timestamp: 2018-04-10T14:15:42.000Z, Cloud C

...

ID: 10400100368A8D00, Timestamp: 2017-12-01T13:08:46.000Z, Cloud Cover: 22, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 1040010035B78300, Timestamp: 2017-12-07T13:05:20.000Z, Cloud Cover: 7, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 1040010032C8C800, Timestamp: 2017-08-05T13:16:08.733Z, Cloud Cover: 13, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 103001006A9F4600, Timestamp: 2017-05-13T12:46:17.000Z, Cloud Cover: 13, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 103001005E0F2C00, Timestamp: 2016-10-08T12:56:06.230Z, Cloud Cover: 11, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 1030010058504400, Timestamp: 2016-06-11T12:47:09.535Z, Cloud Cover: 24, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 1030010057334800, Timestamp: 2016-05-31T12:52:51.469Z, Cloud Cover: 17, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 1040010014C0EF00, Timestamp: 2015-11-25T12:55:34.729Z, Cloud Cover: 7, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 1030010030B47200, Timestamp: 2014-05-30T13:00:48.620Z, Cloud Cover: 8, Image bands: PAN_MS1_MS2, IDAHO: 0
Numb

ID: 103001005CB90D00, Timestamp: 2016-09-03T15:55:54.979Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 103001005A427B00, Timestamp: 2016-08-04T16:02:57.636Z, Cloud Cover: 3, Image bands: PAN_MS1_MS2, IDAHO: 0
Number of catalog IDs not available in IDAHO: 10
Number of catalog IDs available in IDAHO: 0

Number of results remaining: 0
POINT (28.29488109139243 -15.39567695768504)
ID: 104001003A662100, Timestamp: 2018-03-19T08:45:23.000Z, Cloud Cover: 7, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 104001003DCCA600, Timestamp: 2018-05-09T08:56:53.000Z, Cloud Cover: 5, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 104001003E81FB00, Timestamp: 2018-07-04T08:50:11.000Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 104001003E612200, Timestamp: 2018-07-04T08:48:58.000Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 10400100406BB400, Timestamp: 2018-08-05T08:59:33.000Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 104001004099D000, Timestamp: 2018-08-10T08:39:35.000Z, C

ID: 104001003035C500, Timestamp: 2017-07-29T19:17:16.000Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 1040010028028E00, Timestamp: 2017-02-11T19:14:07.867Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 1
ID: 103001006CB6B500, Timestamp: 2017-07-12T18:55:25.000Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 103001006B6CBB00, Timestamp: 2017-06-23T18:56:38.000Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 10300100593C8D00, Timestamp: 2016-06-22T19:07:57.651Z, Cloud Cover: 4, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 1030010057CDF900, Timestamp: 2016-06-22T19:08:10.801Z, Cloud Cover: 9, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 10300100587FBD00, Timestamp: 2016-06-20T18:41:38.448Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 1
ID: 1030010056D66B00, Timestamp: 2016-06-17T18:53:03.394Z, Cloud Cover: 6, Image bands: PAN_MS1_MS2, IDAHO: 0
Number of catalog IDs not available in IDAHO: 7
Number of catalog IDs available in IDAHO: 3

Number of results remaining

ID: 10400100054C7300, Timestamp: 2014-11-25T08:13:33.787Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 103001005ACD6100, Timestamp: 2016-07-26T08:28:22.571Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 0
Number of catalog IDs not available in IDAHO: 10
Number of catalog IDs available in IDAHO: 0

Number of results remaining: 4
POINT (-77.05669246521414 38.84131578046704)
ID: 1040010038C6A100, Timestamp: 2018-02-03T16:32:33.000Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 1
ID: 104001003CC69600, Timestamp: 2018-05-02T16:35:51.000Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 1
ID: 104001003817E000, Timestamp: 2018-02-08T16:13:40.000Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 1
ID: 104001003CB83B00, Timestamp: 2018-05-07T16:15:50.000Z, Cloud Cover: 22, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 10400100367D5500, Timestamp: 2018-01-14T16:14:22.000Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 1
ID: 1040010039662400, Timestamp: 2018-02-27T16:17:14.000Z, 

ID: 1040010041AAFE00, Timestamp: 2018-10-04T00:14:56.000Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 1
ID: 104001003F6B1C00, Timestamp: 2018-07-08T00:12:12.000Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 1040010041577E00, Timestamp: 2018-09-15T00:11:54.000Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 1040010042761700, Timestamp: 2018-10-10T00:10:48.000Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 104001004148F600, Timestamp: 2018-09-15T00:12:55.000Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 0
Number of catalog IDs not available in IDAHO: 9
Number of catalog IDs available in IDAHO: 1

Number of results remaining: 3

Process complete


In [14]:
output.to_csv(os.path.join(pth, outname))

### Order any missing images

In [22]:
order_df = output.loc[output.On_IDAHO == 0]
order_IDs = list(order_df.ID.values)
for i in order_IDs:
    gbdx.ordering.order(i)

### Download images to drive

In [33]:
sys.path.insert(0, r"C:\Users\charl\Documents\GitHub\GOST_GBDX")
from GOST_GBDx_Tools import gbdxTasks
from GOST_GBDx_Tools import gbdxURL_misc
from gbdxtools import CatalogImage

curTasks = gbdxTasks.GOSTTasks(gbdx)
gbdxUrl = gbdxURL_misc.gbdxURL(gbdx)

curFolder=r'D:\WWTP\GBDX_images\MechaTurk'

for root, dirs, files in os.walk(curFolder, topdown=False):
    pass

output = pd.read_csv(os.path.join(curFolder, outname))

print('-- Off %s images in the input .csv, %s are currently on IDAHO, and %s need to be ordered or uploaded --' % (orig, curr, orig-curr))

-- Off 544 images in the input .csv, 133 are currently on IDAHO, and 411 need to be ordered or uploaded --


In [37]:
for root, dirs, files in os.walk(curFolder, topdown=False):
    pass

i = 0
for i in range(0, len(output)):
    
        cWKT = loads(output.AOI_WKT.iloc[i])
        AOI_ID = output.AOI_ID.iloc[i]
        catID = output.ID.iloc[i]
        curFile = 'img_%s_%s.tif' % (AOI_ID,catID)
         
        a = os.path.join(curFolder, curFile)

        if curFile in files:
            print('file already exists, skipping download: %s' % curFile)
            pass
        else:
            print('downloading: %s' % curFile)
            try:
                curTasks.downloadImage(catID, a, boundingBox=cWKT.bounds, imgChipSize = 100000)
            except:
                print('cannot download image: %s' % curFile)
                pass

file already exists, skipping download: img_1_1040010009701200.tif
file already exists, skipping download: img_2_1040010014BC0800.tif
file already exists, skipping download: img_3_104001003B838B00.tif
file already exists, skipping download: img_4_1040010027AF2C00.tif
file already exists, skipping download: img_5_104001003A250100.tif
file already exists, skipping download: img_6_104001003E675D00.tif
file already exists, skipping download: img_7_103001006402FB00.tif
file already exists, skipping download: img_8_1030010078762100.tif
file already exists, skipping download: img_9_104001003F9AF100.tif
file already exists, skipping download: img_10_103001006B3D7400.tif
file already exists, skipping download: img_11_104001003FAD2E00.tif
file already exists, skipping download: img_12_104001003B254500.tif
file already exists, skipping download: img_13_104001002F6C1100.tif
file already exists, skipping download: img_14_1040010032BC3B00.tif
file already exists, skipping download: img_16_1030050042

cannot download image: img_231_104001000ACF7C00.tif
file already exists, skipping download: img_232_1040050002DBD000.tif
file already exists, skipping download: img_233_104001003F8D5E00.tif
file already exists, skipping download: img_234_104001000346E400.tif
file already exists, skipping download: img_235_1030050074F8D200.tif
file already exists, skipping download: img_236_104001003A98C400.tif
file already exists, skipping download: img_237_104001000E90AA00.tif
file already exists, skipping download: img_238_10300100872EF700.tif
file already exists, skipping download: img_239_104001002C7D8200.tif
file already exists, skipping download: img_240_104001001DCAB800.tif
file already exists, skipping download: img_241_1040010028408E00.tif
file already exists, skipping download: img_242_1040010038A17100.tif
file already exists, skipping download: img_243_10400100422EC600.tif
file already exists, skipping download: img_245_10300100556C1200.tif
file already exists, skipping download: img_246_104

file already exists, skipping download: img_608_104001003B126300.tif
file already exists, skipping download: img_609_1040010041AAFE00.tif
file already exists, skipping download: img_610_1040010032625F00.tif
file already exists, skipping download: img_611_104001000D6E2100.tif
file already exists, skipping download: img_613_104001003E81FB00.tif
file already exists, skipping download: img_614_1030010055353800.tif
file already exists, skipping download: img_615_1040010041AAFE00.tif
file already exists, skipping download: img_616_10300100844FE200.tif
file already exists, skipping download: img_617_104001002FC52A00.tif
file already exists, skipping download: img_618_10400100422EC600.tif
downloading: img_619_1040010028028E00.tif
cannot download image: img_619_1040010028028E00.tif
file already exists, skipping download: img_620_103001005D12BE00.tif
file already exists, skipping download: img_621_1030010078D36500.tif
file already exists, skipping download: img_622_104001004050C300.tif
file alre